In [1]:
# Packages to be used
import networkx as nx
from networkx.algorithms import approximation as approx
from networkx.algorithms import community
import numpy as np
import matplotlib.pyplot as plt
import ndlib.models.epidemics as ep
import ndlib.models.ModelConfig as mc
import random
import scipy
import math
import sys
import pandas as pd
import timeit
import community
# from time import sleep
from igraph import *
import networkx.algorithms.community as nx_comm
from networkx.algorithms import *


# G = nx.erdos_renyi_graph(200, 0.1, directed = True)
# print(G)


In [3]:
# 75888 508837
# G = nx.read_edgelist("Wiki-Vote.txt", create_using=nx.DiGraph(), nodetype=int)
G1 = nx.read_edgelist("Wiki-Vote.txt", create_using=nx.Graph(), nodetype=int)

In [8]:
G2 = nx.read_edgelist("facebook_combined.txt", create_using=nx.Graph(), nodetype=int)

In [ ]:
for u,v in G.edges():
    G[u][v]['weight'] = 1/G.in_degree(v)
G.get_edge_data(30,1412)
# G.in_degree(1412)


In [ ]:
lc = nx.community.louvain_communities(G)
# print(lc[1])


In [ ]:
communities = list(nx.community.greedy_modularity_communities(G))

In [ ]:
min_community_size = min(len(community) for community in communities)

# min_community_size = min(len(community) for community in lc)

In [ ]:
small_community_nodes = set(node for community in communities
                            if len(community) == min_community_size
                            for node in community)

# small_community_nodes = set(node for community in lc
#                             if len(lc) == min_community_size
#                             for node in community)



In [ ]:
print(len(small_community_nodes))

In [ ]:
max_degree_node = max(small_community_nodes, key=G.degree)

In [ ]:
seeds = set([max_degree_node])
while True:
    nodes = set(G)
    nodes -= seeds
    if not nodes:
        break
    node_scores = [(node, min([len(set(G.neighbors(n))) for n in seeds | set([node])]))
                   for node in nodes]
    node_scores.sort(key=lambda x: x[1], reverse=True)
    if node_scores[0][1] == node_scores[-1][1]:
        break
    seeds.add(node_scores[0][0])
print(seeds)

In [5]:
bc = nx.betweenness_centrality(G)
sorted_nodes = sorted(bc.items(), key=lambda item: item[1], reverse=True)
num = 10
fairness_allocation = {}
for i, (node, bc_value) in enumerate(sorted_nodes):
    allocation = (1/(i+1)) * (num - len(fairness_allocation))
    fairness_allocation[node] = allocation
    if len(fairness_allocation) == num:
        break
print(fairness_allocation)
keyslist = list(fairness_allocation.keys())
print(keyslist)

{2565: 10.0, 1549: 4.5, 15: 2.6666666666666665, 72: 1.75, 737: 1.2000000000000002, 1166: 0.8333333333333333, 5079: 0.5714285714285714, 2328: 0.375, 2237: 0.2222222222222222, 28: 0.1}


In [ ]:
undir_betweenness = nx.betweenness_centrality(G1)

Finding seeds using louvain communities and using the top node from each community

In [18]:
import community
partition = community.best_partition(G_undir)

community_sizes = {}
for node, community_id in partition.items():
    community_sizes[community_id] = community_sizes.get(community_id, 0) + 1

# compute the average size of a communtiy
avg_community_size = sum(community_sizes.values()) / len(community_sizes)

# sort the communitities by size in descending order
sorted_communities = sorted(community_sizes.items(), key=lambda x: x[1], reverse=True)

# initialie list of seeds nodes
seeds =[]

# Iterate over the communities and add the top nodes to the seeds list
for community_id, size in sorted_communities:
    nodes = [node for node, c_id in partition.items() if c_id == community_id]
    nodes_sorted = sorted(nodes, key=lambda n: G_undir.degree(n), reverse=True)
    community_seed_count = int(round(size / avg_community_size))
    seeds += nodes_sorted[:community_seed_count]


In [19]:
print(seeds)

[11, 1151, 1608, 789, 993, 1297, 311, 173, 6, 766, 457, 2688, 3456, 15, 5079, 1133, 2972, 1549, 1166, 3352, 2485, 2328, 2871, 2651, 2237, 2565, 1374, 5524, 4037, 737, 5802, 2658]


Use louvain communities with equality fairness and betweenness centrality

In [24]:
import community

# Partition into communities
partition = community.best_partition(G1)
print(len(partition))
# betweenness_centralities = {}
closeness_centralities = {}
degree_centralities = {}
for community_id in set(partition.values()):
    community_nodes = [n for n in partition.keys() if partition[n] == community_id]
    community_subgraph = G1.subgraph(community_nodes)
    # betweenness_centralities[community_id] = nx.betweenness_centrality(community_subgraph)
    # degree_centralities[community_id] = nx.degree_centrality(community_subgraph)
    closeness_centralities[community_id] = nx.closeness_centrality(community_subgraph)

# Calculate fairness metric
fairness_metrics = {}
for community_id in set(partition.values()):
    # fairness = max(betweenness_centralities[community_id].values()) - min(betweenness_centralities[community_id].values())
    # fairness = max(degree_centralities[community_id].values()) - min(degree_centralities[community_id].values())
    fairness = max(closeness_centralities[community_id].values()) - min(closeness_centralities[community_id].values())                                                                        
    fairness_metrics[community_id] = fairness
    
# Calculate seeds from each community
seeds = []
min_fairness_community = min(fairness_metrics, key=fairness_metrics.get)

for community_id in set(partition.values()):
    community_nodes = [n for n in partition.keys() if partition[n] == community_id]
    community_subgraph = G1.subgraph(community_nodes)

    if community_id == min_fairness_community:
        # node = max(betweenness_centralities[community_id], key=betweenness_centralities[community_id].get)
        # node = max(degree_centralities[community_id], key=degree_centralities[community_id].get)
        node = max(closeness_centralities[community_id], key=closeness_centralities[community_id].get)

    else:
        # nodes_sorted = sorted(betweenness_centralities[community_id].items(), key=lambda x: x[1], reverse=True)
        # nodes_sorted = sorted(degree_centralities[community_id].items(), key=lambda x: x[1], reverse=True)
        nodes_sorted = sorted(closeness_centralities[community_id].items(), key=lambda x: x[1], reverse=True)
        node = nodes_sorted[0][0]

    seeds.append(node)

7115


In [26]:
num_distinct_values = len(set(partition.values()))

print("The dictionary has", num_distinct_values, "distinct values.")

The dictionary has 31 distinct values.


In [25]:
print(seeds)

[11, 2565, 5524, 1166, 15, 6691, 2304, 5231, 3194, 3244, 8076, 4168, 4540, 4544, 5413, 5678, 5766, 5970, 6025, 6089, 6100, 6258, 6266, 7033, 7190, 7194, 7467, 7494, 7972, 7981, 8014]


In [ ]:
# This code can be used with the previous two cell blocks to calculate the influence count. 
# It combines the method of calculating the spread using spreading model as defined in the NDlib package, witht the different heuristic approaches.
for index in range(4):
    # model = ep.IndependentCascadesModel(G1)
    model = ep.ThresholdModel(G1)
    cfg = mc.Configuration()
    # cfg.add_model_parameter('beta', 0.01)
    # cfg.add_model_parameter('lambda', 0.005)
    infected_nodes = seeds
    cfg.add_model_initial_configuration("Infected", infected_nodes)
    # cfg.add_model_parameter("fraction_infected", 0.05)
    model.set_initial_status(cfg)
    # threshold = 0.2
    # for e in G1.edges():
    #     cfg.add_edge_configuration("threshold", e, random.uniform(0,1))
    for n in G1.nodes():
        cfg.add_node_configuration("threshold", n, random.uniform(0, 1))

    # iterations = model.iteration_bunch(500, True, True)
    # trends = model.build_trends(iterations)
    # viz = DiffusionTrend(model, trends)
    # p = viz.plot(width=800, height=800)
    # show(p)
    list = []
    for i in range(0,1000):
        iterations = model.iteration_bunch(1)

        dict = iterations[0]
        for i in dict:
            k = (i,dict[i])
            list.append(k)
    list2 = []
    for i in range(len(list)):
        if list[i][0] == 'status':
            list2.append(list[i][1])
    list4 = []
    for t in range(len(list2)):
        for f in list2[t]:
            if list2[t][f] == 1:
                list4.append(f)
    print("================================")
    print(f"Results for iteration {index+1}:")
    print("Total nodes infected: ", len(list4), "out of", len(G1.nodes()))
    print("Percentage of total infected nodes with current seed set: ", (len(list4)/len(G1.nodes()))*100, "%")
    print("================================")





In [ ]:
len(nx_comm.louvain_communities(g, seed = None))

In [ ]:
# g = nx.read_edgelist("Wiki-Vote.txt", create_using=nx.DiGraph(), nodetype= int)
model = ep.IndependentCascadesModel(G)
config = mc.Configuration()
infected_nodes = result
config.add_model_initial_configuration("Infected", infected_nodes)
threshold = 0.1
for e in G.edges():
    config.add_edge_configuration("threshold", e, random.uniform(0,1))
model.set_initial_status(config)
iterations = model.iteration_bunch(500,False,True)
trends = model.build_trends(iterations)

In [ ]:
from bokeh.io import output_notebook, show
from ndlib.viz.bokeh.DiffusionTrend import DiffusionTrend

viz = DiffusionTrend(model, trends)
p = viz.plot(width=600, height=600)
show(p)

In [6]:
import community
import itertools

new_partition = community.best_partition(G1)
betweenness = nx.betweenness_centrality(G1)

min_betweenness = {}
for node, community_id in new_partition.items():
    if community_id not in min_betweenness or betweenness[node] < min_betweenness[community_id]:
        min_betweenness[community_id] = betweenness[node]

candidate_seeds = []

for community1, community2 in itertools.combinations(set(new_partition.values()),2):
    max_betweenness = -1
    max_node = None
    for node in G1.nodes():
        if new_partition[node] == community1 and node not in candidate_seeds:
            if betweenness[node] > max_betweenness:
                max_betweenness = betweenness[node]
                max_node = node
    
    if max_betweenness >= min_betweenness[community2]:
        candidate_seeds.append(max_node)
    if len(candidate_seeds) == 28:
        break

print(candidate_seeds)

[4167, 4168, 4540, 4541, 2565, 766, 2237, 2688, 2328, 3352, 2398, 2485, 1133, 2871, 2654, 4191, 8293, 2535, 2651, 68, 4261, 1305, 3028, 988, 825, 4247, 2790, 2660]
